In [ ]:
import numpy as np
import random
import copy
import time
import math

In [7]:
class AG_Himmelblau():
    def __init__(self, N: int):
        self.population = [[math.trunc(random.uniform(-5, 5) * 100) / 100 for _ in range(2)] for i in range(N)]
        self.fitnesses = []

    def sort_pop(self, fitness_function, reverse_sort: bool) -> tuple[list[list], list]:
        """Sort population by fitness function. Return tuple with population list and fitness list"""

        fitness_list = [fitness_function(ind) for ind in self.population]
        lista = sorted(zip(self.population, fitness_list), key=lambda x: x[1], reverse=reverse_sort)
        self.population = [x[0] for x in lista]
        self.fitnesses = [x[1] for x in lista]

    def select(self, T: int) -> list[float, float]:
        """Return a copy of an indivudual by tournament selection. Population already ordered by fitness"""

        choices=random.choices(copy.copy(self.population), k=T)
        indices=[self.population.index(c) for c in choices]
        return self.population[np.argmin(indices)]

    def crossover(self, parent1: list[float, float], parent2: list[float, float], pcross: float) -> tuple[list,list]: 
        if random.random()<pcross:
            child1 = [parent1[0], parent2[1]]
            child2 = [parent1[1], parent2[0]]
        else:
            child1, child2 = parent1[:], parent2[:]
        return child1, child2

    def mutate(self, individual: list[float, float], pmut: float) -> list[float, float]:
        if random.random()<pmut:
            individual[0], individual[1] = [math.trunc(random.uniform(-5, 5) * 100) / 100 for _ in range(2)]
        return individual

    def evolve(self, fitness_function, pmut=0.1, pcross=0.7, ngen=100, T=2, trace=50, reverse_sort=False, elitism=False) -> None:
        """Evolution procedure. Initial population already created"""

        for i in range(ngen):
            new_pop = []
            self.sort_pop(fitness_function, reverse_sort)
            if elitism:
                new_pop.append(self.population[0])
                new_pop.append(self.population[1])
            while len(new_pop) != 100:   
                individual1 = self.select(T)
                individual2 = self.select(T)
                child1, child2 = self.crossover(individual1, individual2, pcross)
                mutated1 = self.mutate(child1, pmut)
                mutated2 = self.mutate(child2, pmut)
                new_pop.append(mutated1)
                new_pop.append(mutated2)
                
            self.population = [*new_pop] # make a copy

            if i % trace == 0 or i == ngen-1: # en la última gen se ordena
                self.sort_pop(fitness_function, reverse_sort)
                print(f"Nº gen: {i}, Best ind: {self.population[0]}, Best fitness: {self.fitnesses[0]:.3f}")

In [8]:
# himmelblau modified to have only one global minimum (-3.77, -3.28)
# si se pone "onlyone" a True 
onlyone = True
def himmelblau(chromosome: list[float, float]) -> float:
	x = chromosome[0]
	y = chromosome[1]
	fxy = (x**2 + y - 11)**2 + (x + y**2 -7)**2
	if onlyone and (x>0 or y>0): 
		fxy += 0.5
	return fxy

# fitness para himmelblau: valor mínimo de la función
def fitness_himmel(chromosome: list[float, float]):
	return 1 / (1 + himmelblau(chromosome))

## **---------------------------Tests---------------------------**

In [26]:
start = time.time()
genetic_algorithm = AG_Himmelblau(N=100) 
final_population = genetic_algorithm.evolve(fitness_function=fitness_himmel, pmut=0.1, ngen=1000, T=6, trace=100, reverse_sort=True)
minutos, segundos = divmod(time.time()-start, 60)
print(f"*******Tiempo transcurrido: {int(minutos)} minutos y {segundos:.2f} segundos*******")

Nº gen: 0, Best ind: [3.51, -1.94], Best fitness: 0.510
Nº gen: 100, Best ind: [-3.71, -3.27], Best fitness: 0.796
Nº gen: 200, Best ind: [-3.71, -3.27], Best fitness: 0.796
Nº gen: 300, Best ind: [-3.71, -3.27], Best fitness: 0.796
Nº gen: 400, Best ind: [-3.78, -3.29], Best fitness: 0.998
Nº gen: 500, Best ind: [-3.78, -3.29], Best fitness: 0.998
Nº gen: 600, Best ind: [-3.78, -3.29], Best fitness: 0.998
Nº gen: 700, Best ind: [-3.78, -3.29], Best fitness: 0.998
Nº gen: 800, Best ind: [-3.78, -3.29], Best fitness: 0.998
Nº gen: 900, Best ind: [-3.78, -3.29], Best fitness: 0.998
Nº gen: 999, Best ind: [-3.78, -3.29], Best fitness: 0.998
*******Tiempo transcurrido: 0 minutos y 0.66 segundos*******
